## Load corpus


In [1]:
import os
from tqdm import tqdm
from corpus import download_and_unzip, catgeories, read_text_files, corpus_root

download_and_unzip()

Already downloaded and extracted!


In [2]:
reviews = []
labels = []

# we can't use the previous tokenizers here
# idx 0 -> neg, 1 -> pos
for idx, cat in enumerate(catgeories):
    path = os.path.join(corpus_root, cat)
    texts = read_text_files(path)

    for i in tqdm(range(len(texts)), desc="prepare_corpus"):
        text = texts[i]
        reviews.append(text)
        labels.append(idx)

print()
print(len(reviews))
print(len(labels))

prepare_corpus: 100%|██████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2748560.94it/s]


2000
2000


## Tokenizer
https://huggingface.co/microsoft/xtremedistil-l6-h384-uncased

In [3]:
from transformers import AutoTokenizer
import torch

model_name = "microsoft/xtremedistil-l6-h384-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name) 

## Split corpus


In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    reviews, labels, random_state=42, train_size=0.8
)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=42)

## Dataloaders

In [5]:
import torch
from torch.utils.data import Dataset

# custom dataset
class PolarityReviewDataset(Dataset):

    def __init__(self, reviews, labels, tokenizer):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        label = self.labels[idx]

        # encode review text
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            return_token_type_ids=False,
            pad_to_max_length="max_len",
            return_attention_mask=True,
            return_tensors="pt"
        )

        return {
            "text": review,
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding['attention_mask'].flatten(),
            "label": torch.tensor(label)
        }

training_dataset = PolarityReviewDataset(x_train, y_train, tokenizer)
val_dataset = PolarityReviewDataset(x_val, y_val, tokenizer)

In [6]:
from torch.utils.data import DataLoader
batch_size = 32

# loader from custom dataset
train_loader = DataLoader(training_dataset, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)


## Classifier

In [7]:
from transformers import AutoModel
import torch.nn as nn
import torch.nn.functional as F

class SentiBERT(nn.Module):
    def __init__(self, model_name=model_name):
        super(SentiBERT, self).__init__()

        self.bert = AutoModel.from_pretrained(model_name)
        self.linear = nn.Linear(384, 1) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        out = out.pooler_output
        
        out = self.linear(out)
        out = self.sigmoid(out)

        return out

In [8]:
senti_bert = SentiBERT()

## Setup device

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Hyperparams

In [10]:
learning_rate = 2e-5

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(senti_bert.parameters(), lr=learning_rate)

## Send to device

In [11]:
senti_bert = senti_bert.to(device)

## Train

In [12]:
import numpy as np

epochs = 5

def train(model, train_loader, val_loader, epochs, optimizer, loss_fn):
    print_counter = 0 # print loss for each 10th count

    for e in tqdm(range(epochs), desc=f"train_sentibert_for_{epochs}_epochs"):
        model.train()
        for td in train_loader:
            print_counter += 1

            # unpack data
            input_ids = td["input_ids"]
            input_ids = input_ids.to(device)

            attention_mask = td["attention_mask"].to(device)
            attention_mask = attention_mask.to(device)

            label = td["label"]
            label = label.long().to(device)


            # zero gradients
            model.zero_grad()

            # forward pass
            output = model(input_ids, attention_mask)
            # the max probability based class
            output, _ = torch.max(output, dim=1)

            # backprop
            loss = loss_fn(output, label.float())
            loss.backward()
            
            #clip gradients
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()

            # log loss 
            if print_counter % 10 == 0:
                validation_losses = []
                
                model.eval() # switch mode
                with torch.no_grad():
                    for td in val_loader:
                        # unpack data
                        # unpack data and send to device
                        input_ids = td["input_ids"]
                        input_ids = input_ids.to(device)

                        attention_mask = td["attention_mask"]
                        attention_mask = attention_mask.to(device)

                        label = td["label"]
                        label = label.long().to(device)
                        
                        # repeat same steps from forward pass
                        out = model(input_ids, attention_mask)
                        out, _ = torch.max(out, dim=1)
                        val_loss = loss_fn(out, label.float())
                        
                        # add loss to validation losses
                        validation_losses.append(val_loss.item())
                    print(f"\nEpoch: {e + 1}/{epochs}\tStep: {print_counter}\tTrain Loss: {loss.item()}\tValidation Loss: {np.mean(validation_losses)}")

                model.train()


%time train(model=senti_bert, train_loader=train_loader, val_loader=val_loader, epochs=epochs, optimizer=optimizer, loss_fn=loss_fn)

train_sentibert_for_5_epochs:   0%|                                                   | 0/5 [00:00<?, ?it/s]/home/shawon/anaconda3/envs/experiments/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(



Epoch: 1/5	Step: 10	Train Loss: 0.6892192363739014	Validation Loss: 0.6876241207122803

Epoch: 1/5	Step: 20	Train Loss: 0.663413405418396	Validation Loss: 0.667889392375946

Epoch: 1/5	Step: 30	Train Loss: 0.6018240451812744	Validation Loss: 0.622662490606308


train_sentibert_for_5_epochs:  20%|████████▌                                  | 1/5 [00:12<00:49, 12.31s/it]


Epoch: 1/5	Step: 40	Train Loss: 0.5341616868972778	Validation Loss: 0.5558145880699158

Epoch: 2/5	Step: 50	Train Loss: 0.4293751120567322	Validation Loss: 0.5041411936283111

Epoch: 2/5	Step: 60	Train Loss: 0.4341110587120056	Validation Loss: 0.47545974552631376

Epoch: 2/5	Step: 70	Train Loss: 0.4636622667312622	Validation Loss: 0.452631288766861


train_sentibert_for_5_epochs:  40%|█████████████████▏                         | 2/5 [00:24<00:36, 12.00s/it]

...


Epoch: 3/5	Step: 120	Train Loss: 0.2941393256187439	Validation Loss: 0.44250090420246124

Epoch: 4/5	Step: 130	Train Loss: 0.3219050168991089	Validation Loss: 0.42692846059799194

Epoch: 4/5	Step: 140	Train Loss: 0.36851218342781067	Validation Loss: 0.42613582909107206

Epoch: 4/5	Step: 150	Train Loss: 0.23285815119743347	Validation Loss: 0.41176736950874326


train_sentibert_for_5_epochs:  80%|██████████████████████████████████▍        | 4/5 [00:47<00:11, 11.87s/it]


Epoch: 4/5	Step: 160	Train Loss: 0.21287108957767487	Validation Loss: 0.45053770840168

Epoch: 5/5	Step: 170	Train Loss: 0.17056509852409363	Validation Loss: 0.40660481452941893

Epoch: 5/5	Step: 180	Train Loss: 0.4009157419204712	Validation Loss: 0.41614746749401094

Epoch: 5/5	Step: 190	Train Loss: 0.23759794235229492	Validation Loss: 0.40654144287109373


train_sentibert_for_5_epochs: 100%|███████████████████████████████████████████| 5/5 [00:59<00:00, 11.90s/it]


Epoch: 5/5	Step: 200	Train Loss: 0.2404182404279709	Validation Loss: 0.4164420336484909
CPU times: user 59.9 s, sys: 352 ms, total: 1min
Wall time: 59.5 s


## Inference

In [13]:
# test data is a list of reviews as strings
def classify_sentiment(model, test_data, tokenizer):
    prediction = []
    # switch model mode
    model.eval()
    with torch.no_grad():

        for i in tqdm(range(len(test_data)), desc="inference"):
            review = test_data[i]
            
            # encode data
            encoded = tokenizer.encode_plus(
                review,
                add_special_tokens=True,
                max_length=512,
                truncation=True,
                return_token_type_ids=False,
                pad_to_max_length="max_len",
                return_attention_mask=True,
                return_tensors="pt"
            )
            
            # unpack
            input_ids = encoded["input_ids"].to(device)
            attention_mask = encoded["attention_mask"].to(device)
            
            
            # forward pass
            pred = model(input_ids, attention_mask)
            pred, _ = torch.max(pred, dim=1)
            # round to the nearest integer
            pred =  torch.round(pred.squeeze())
            
            
            # add to list
            prediction.append(pred.cpu().detach().numpy()) # dear pytorch team, find a easier wrapper please!
            
    return np.array(prediction)

In [14]:
y_pred = classify_sentiment(senti_bert, x_test, tokenizer)

inference: 100%|█████████████████████████████████████████████████████████| 400/400 [00:01<00:00, 222.02it/s]


In [15]:
y_pred = y_pred.reshape(-1, 1)

In [16]:
y_test = np.array(y_test).reshape(-1, 1)

## Evaluation

In [17]:
from sklearn.metrics import classification_report

print(classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           0       0.79      0.94      0.86       199
           1       0.93      0.75      0.83       201

    accuracy                           0.84       400
   macro avg       0.86      0.85      0.84       400
weighted avg       0.86      0.84      0.84       400

